In [3]:
import findspark
findspark.init()

In [2]:
spark_xml='/home/sarbajit/Documents/Programs/spark/jars/spark-xml_2.12-0.12.0.jar'
xmlschemaPath = '/home/sarbajit/Documents/Programs/spark/jars/XmlSchema-1.4.7.jar'

In [1]:
!conda install pydeequ

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - pydeequ

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [5]:
from pyspark.sql import SparkSession, Row
# spark = SparkSession.builder \
#         .appName("XMLProcessor") \
#         .config("spark.jars",'/home/sarbajit/Documents/Programs/spark/jars') \
#         .config("spark.executor.extraClassPath",spark_xml + ":" + xmlschemaPath) \
#         .config("spark.executor.extraLibrary",spark_xml) \
#         .config("spark.driver.extraClassPath",spark_xml + ":" + xmlschemaPath) \
#         .getOrCreate()

# spark = SparkSession.builder.appName("XmlProcessor").getOrCreate()

import pydeequ

spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

df = spark.sparkContext.parallelize([
            Row(a="foo", b=1, c=5),
            Row(a="bar", b=2, c=6),
            Row(a="baz", b=3, c=None)]).toDF()
df.show()

Please set env variable SPARK_VERSION
22/01/27 15:40:37 WARN Utils: Your hostname, sarbajit-laptop resolves to a loopback address: 127.0.1.1; using 192.168.0.130 instead (on interface wlp2s0)
22/01/27 15:40:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/sarbajit/.ivy2/cache
The jars for the packages stored in: /home/sarbajit/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c30564dd-60be-4bbd-acfb-e59b85882274;1.0
	confs: [default]
	found com.amazon.deequ#deequ;1.2.2-spark-3.0 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found org.scala-lang#scala-reflect;2.12.1 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in centr

+---+---+----+
|  a|  b|   c|
+---+---+----+
|foo|  1|   5|
|bar|  2|   6|
|baz|  3|null|
+---+---+----+



In [6]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+--------+----+-----+
| entity|instance|name|value|
+-------+--------+----+-----+
|Dataset|       *|Size|  3.0|
+-------+--------+----+-----+



In [ ]:
file_path = '/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/Book (copy).xml'

# df = spark.read.format('xml').options(rowTag='book', attributePrefix='Attr_', ).load(file_path)
df = spark.read.format('xml').options(rowTag='catalog', rowValidationXSDPath="/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/BookXMLSchema2.xsd", inferSchema=True).load(file_path)
df.show()

df.printSchema()

In [4]:
from pyspark.java_gateway import java_import
java_import(spark.sparkContext._gateway.jvm, "com.databricks.spark.xml.util.XSDToSchema,java.nio.file.Paths")
jvm = spark.sparkContext._gateway.jvm

In [5]:
# obj = jvm.java.nio.file.Paths.get('/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/BookXMLSchema.xsd')
schema = jvm.com.databricks.spark.xml.util.XSDToSchema.read('/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/BookXMLSchema.xsd')
schema

Py4JJavaError: An error occurred while calling z:com.databricks.spark.xml.util.XSDToSchema.read.
: java.lang.NoSuchMethodError: org.apache.ws.commons.schema.XmlSchemaCollection.read(Ljava/io/Reader;)Lorg/apache/ws/commons/schema/XmlSchema;
	at com.databricks.spark.xml.util.XSDToSchema$.read(XSDToSchema.scala:78)
	at com.databricks.spark.xml.util.XSDToSchema.read(XSDToSchema.scala)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.

In [ ]:
from pyspark.sql.functions import col,explode_outer

dfout = df.select(col('catalog.*'), col('family.*'), "*")
dfout.show()

In [ ]:
dfout.show()
dfout= dfout.withColumn('book',explode_outer(col('book')))
dfout=dfout.withColumn('member',explode_outer(col('member')))
# dfout=dfout.select('book','member')
dfout.show()

In [ ]:
dfout = dfout.select ('book.*','member.*')
dfout.show()


In [ ]:
dfout.columns

In [ ]:
dataMap = {
    'catalog.book._id':'bookId',
    'catalog.book.author' : 'author',
    'catalog.book.title' : 'title',
    'catalog.book.genre' : 'genre',
    'catalog.book.price' : 'price',
    'catalog.book.publish_date' : 'publish_date',
    'family.member._id': 'member_id',
    'family.member.firstname': 'firstname',
    'family.member.lastname': 'lastname',
    'family.member.age': 'age',
    'family.member.profession': 'profession',
    'family.member.price': 'price',
    'family.member.sex': 'sex',
}

In [ ]:
av=[]

for node in dataMap.keys():
    try:
        df.select(col(node))
        av.append(node)
    except:
        pass

print(av)


In [ ]:
colList = []
for each in av:
    colList.append(col(each).alias(dataMap[each]))

df.select(*colList).show()

In [ ]:
import xmlschema
schema_details = xmlschema.XMLSchema('/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/BookXMLSchema.xsd')

# print(schema.attribute_form_default)

In [ ]:
from py4j.java_gateway import java_import
jvm = spark.sparkContext._gateway.jvm
java_import(jvm,'com.databricks.spark.xml.util.*')

schema = jvm.XSDToSchema('/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/BookXMLSchema.xsd')
print(schema)

In [ ]:
xml_df=spark.createDataFrame(schema=schema_details, data=[])
xml_df.show()

In [ ]:
for node in schema.iter_components():
    print(node)

In [ ]:
data = schema.decode("/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/Book.xml")


In [ ]:
import json 

with open('/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/data.json','w') as fp:
    json.dump(data, fp, indent=4)


In [ ]:
node = schema.complex_types[0]
isinstance(node,xmlschema.validators.complex_types.XsdComplexType)


In [ ]:
go(node)

In [ ]:
schema.iter

In [ ]:
df.select(*['author','description']).show()

In [ ]:
sc = spark.sparkContext

rdd1 = sc.textFile('/home/sarbajit/Documents/Programs/spark/Data/imp/Emp.txt')
print(rdd1.collect())

In [ ]:
import xmlschema
xsdFile = "/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/BookXMLSchema.xsd"
schema = xmlschema.XMLSchema(xsdFile)

In [ ]:
with open("dir", 'w') as fp:
    fp.write(str(dir(schema)))

In [ ]:
xmlFile= '/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/Book.xml'
data = schema.to_dict(xmlFile)
print(data)


In [ ]:
schema.decode(xmlFile)

In [62]:
from pyspark.sql.types import StringType,StructType, ArrayType,StructField,DateType
xsdFile = "/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/BookXMLSchema.xsd"
xmlFile= '/home/sarbajit/Documents/Programs/spark/Data/xml/BookDetails/Book.xml'


In [64]:
df = spark.read.format('xml').options(rowTag='book',inferSchema=True).load(xmlFile)
# df.show()
schema2=df.schema
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- description: struct (nullable = true)
 |    |-- hold: struct (nullable = true)
 |    |    |-- holdingDetails: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- a: long (nullable = true)
 |    |    |    |    |-- b: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: double (nullable = true)
 |-- publish_date: date (nullable = true)
 |-- title: string (nullable = true)



In [65]:
df.select('_id', explode_outer('description.hold.holdingDetails')).select("_id",'col.*').show()

+-----+----+----+
|  _id|   a|   b|
+-----+----+----+
|bk101| 123| abc|
|bk101| 123| abc|
|bk102| 123| abc|
|bk102| 234|abc1|
|bk103|null|null|
|bk104|null|null|
|bk105|null|null|
|bk106|null|null|
|bk107|null|null|
|bk108|null|null|
|bk109|null|null|
|bk110|null|null|
|bk111|null|null|
|bk112|null|null|
+-----+----+----+



In [26]:
schema = StructType([
    StructField('_id', StringType(),True),
    StructField('author', StringType(),True),
    StructField('title', StringType(),True),
    StructField('genre', StringType(),True),
    StructField('price', StringType(),True),
    StructField('publish_date', DateType(),True),
    StructField('description', StructType([
        StructField("_VALUE",StringType(),True)
    ]),True),
])

df = spark.read.format('xml').options(rowTag='book').load(xmlFile, schema=schema)
df.show()
df.schema

+-----+--------------------+--------------------+---------------+-----+------------+-----------+
|  _id|              author|               title|          genre|price|publish_date|description|
+-----+--------------------+--------------------+---------------+-----+------------+-----------+
|bk101|Gambardella, Matthew|XML Developer's G...|       Computer|44.95|        null|     {null}|
|bk102|          Ralls, Kim|       Midnight Rain|        Fantasy| 5.95|        null|     {null}|
|bk103|         Corets, Eva|     Maeve Ascendant|        Fantasy| 5.95|        null|     {null}|
|bk104|         Corets, Eva|     Oberon's Legacy|        Fantasy| 5.95|        null|     {null}|
|bk105|         Corets, Eva|  The Sundered Grail|        Fantasy| 5.95|        null|     {null}|
|bk106|    Randall, Cynthia|         Lover Birds|        Romance| 4.95|        null|     {null}|
|bk107|      Thurman, Paula|       Splish Splash|        Romance| 4.95|        null|     {abcd}|
|bk108|       Knorr, Stefan|  

StructType(List(StructField(_id,StringType,true),StructField(author,StringType,true),StructField(title,StringType,true),StructField(genre,StringType,true),StructField(price,StringType,true),StructField(publish_date,DateType,true),StructField(description,StructType(List(StructField(_VALUE,StringType,true))),true)))

In [73]:
sample = {
    "A":'a1',
    "B.name":"bname",
    "B.dept":"bdept",
    "C.name._id":"cname_id",
    "C.name._VALUE":"cname_value"  ,
    "C.roll._id":"croll_id"  ,
    "C.roll._VALUE":"croll_value"  ,
    'D.person.details.age': "pAge",  
    'D.person.details.sex': "pSex",  
    'D.attach.details.a1': "pOth",  
}

sample2 = {
    "_id":"_id",
    "author":"author",
    "title":"title",
    "genre":"genre",
    "price":"price",
    "publish_date":"publish_date",
    "description":{
        "hold":{
            'holdingDetails':[{'a':'a','b':'b'}]
        }
    }
}


def go(node, r):
    next=set()
    head=dict()
    for k,v in node.items():
        if '.' not in k:
            head[k] = head.get(k,v)
        else:
            k1= k.split(".")[0]
            k2=k.split(k1+'.')[1]
            # print("keys",k1,k2)
            nH=head.get(k1,dict())
            # print(nH)
            nH[k2]=v
            head[k1]=nH
            if '.' in k2:
                next.add(k1)
        # print(head)
    for k in next:
        head[k]=go(head[k],r+1)
    return head

rawSchema=go(sample2,1)
rawSchema




{'_id': '_id',
 'author': 'author',
 'title': 'title',
 'genre': 'genre',
 'price': 'price',
 'publish_date': 'publish_date',
 'description': {'hold': {'holdingDetails': [{'a': 'a', 'b': 'b'}]}}}

In [26]:
rawSchema = {
    "catalog":{"book":[rawSchema]}
    
}
rawSchema

{'catalog': {'book': [{'author': 'author',
    'title': 'title',
    'genre': 'genre',
    'price': 'price',
    'publish_date': 'publish_date',
    'description': {'_id': 'descriptionId', '_VALUE': 'descriptionValue'}}]}}

In [74]:
from pyspark.sql.types import *

def build(node):
    baseSchema = None
    if type(node)==dict:
        inside= []
        for k,v in node.items():
            if type(v)==dict:
                t= build(v)
            elif type(v)==list:
                pass
            else:
                t= StringType()
            inside.append(StructField(k,t,True))
        baseSchema=StructType(inside)
        return baseSchema

    return None        
        
def build2(node):
    baseSchema = None
    if type(node)==dict:
        inside= []
        for k,v in node.items():
            if type(v)==dict:
                t= build2(v)
            elif type(v)==list:
                t = build2(v[0])
                t= ArrayType(t, True)
            else:
                t= StringType()
            inside.append(StructField(k,t,True))
        baseSchema=StructType(inside)
        return baseSchema
    elif type(node)==list:
        t = build2(node[0])
        return ArrayType(t, True)
    else:
        return StringType()

    return None        
schema = build2(rawSchema)
schema.jsonValue()


{'type': 'struct',
 'fields': [{'name': '_id',
   'type': 'string',
   'nullable': True,
   'metadata': {}},
  {'name': 'author', 'type': 'string', 'nullable': True, 'metadata': {}},
  {'name': 'title', 'type': 'string', 'nullable': True, 'metadata': {}},
  {'name': 'genre', 'type': 'string', 'nullable': True, 'metadata': {}},
  {'name': 'price', 'type': 'string', 'nullable': True, 'metadata': {}},
  {'name': 'publish_date', 'type': 'string', 'nullable': True, 'metadata': {}},
  {'name': 'description',
   'type': {'type': 'struct',
    'fields': [{'name': 'hold',
      'type': {'type': 'struct',
       'fields': [{'name': 'holdingDetails',
         'type': {'type': 'array',
          'elementType': {'type': 'struct',
           'fields': [{'name': 'a',
             'type': 'string',
             'nullable': True,
             'metadata': {}},
            {'name': 'b',
             'type': 'string',
             'nullable': True,
             'metadata': {}}]},
          'containsNull':

In [75]:
df = spark.read.format('xml').options(rowTag='book').load(xmlFile, schema=schema)
df.show()
df.printSchema()

+-----+--------------------+--------------------+---------------+-----+------------+--------------------+
|  _id|              author|               title|          genre|price|publish_date|         description|
+-----+--------------------+--------------------+---------------+-----+------------+--------------------+
|bk101|Gambardella, Matthew|XML Developer's G...|       Computer|44.95|  2000-10-01|{{[{123, abc}, {1...|
|bk102|          Ralls, Kim|       Midnight Rain|        Fantasy| 5.95|  2000-12-16|{{[{123, abc}, {2...|
|bk103|         Corets, Eva|     Maeve Ascendant|        Fantasy| 5.95|  2000-11-17|              {null}|
|bk104|         Corets, Eva|     Oberon's Legacy|        Fantasy| 5.95|  2001-03-10|              {null}|
|bk105|         Corets, Eva|  The Sundered Grail|        Fantasy| 5.95|        null|              {null}|
|bk106|    Randall, Cynthia|         Lover Birds|        Romance| 4.95|  2000-09-02|              {null}|
|bk107|      Thurman, Paula|       Splish Spla

In [77]:
df.select('_id',explode_outer("description.hold.holdingDetails")).select('_id',"col.*").show()

+-----+----+----+
|  _id|   a|   b|
+-----+----+----+
|bk101| 123| abc|
|bk101| 123| abc|
|bk102| 123| abc|
|bk102| 234|abc1|
|bk103|null|null|
|bk104|null|null|
|bk105|null|null|
|bk106|null|null|
|bk107|null|null|
|bk108|null|null|
|bk109|null|null|
|bk110|null|null|
|bk111|null|null|
|bk112|null|null|
+-----+----+----+



In [9]:

from pyspark.sql.functions import col, explode_outer

# df.select(explode_outer("catalog.book").alias("book")).select(col("book.*")).show()

In [18]:
df = spark.read.format('xml').options(rowTag='main').load(xmlFile)
df.show()
df.printSchema()

+--------------------+--------------------+--------------------+
|              _xmlns|          _xmlns:xsi|             catalog|
+--------------------+--------------------+--------------------+
|http://www.exampl...|http://www.w3.org...|{[{bk101, Gambard...|
+--------------------+--------------------+--------------------+

root
 |-- _xmlns: string (nullable = true)
 |-- _xmlns:xsi: string (nullable = true)
 |-- catalog: struct (nullable = true)
 |    |-- book: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _id: string (nullable = true)
 |    |    |    |-- author: string (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- description2: string (nullable = true)
 |    |    |    |-- genre: string (nullable = true)
 |    |    |    |-- price: double (nullable = true)
 |    |    |    |-- title: string (nullable = true)



In [ ]:
df = spark.read.format('xml').options(rowTag='main').load(xmlFile)
df.show()
df.printSchema()

+--------------------+--------------------+--------------------+
|              _xmlns|          _xmlns:xsi|             catalog|
+--------------------+--------------------+--------------------+
|http://www.exampl...|http://www.w3.org...|{[{bk101, Gambard...|
+--------------------+--------------------+--------------------+

root
 |-- _xmlns: string (nullable = true)
 |-- _xmlns:xsi: string (nullable = true)
 |-- catalog: struct (nullable = true)
 |    |-- book: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _id: string (nullable = true)
 |    |    |    |-- author: string (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- description2: string (nullable = true)
 |    |    |    |-- genre: string (nullable = true)
 |    |    |    |-- price: double (nullable = true)
 |    |    |    |-- title: string (nullable = true)



In [49]:
s = [['123',4,5,6,], ['123',4,5,7,],['234',4,5,6,],['345',4,5,6,] ,['456',4,5,6,] ]
p = [['123',4,5,6,], ['123',4,5,7,],['234',4,5,6,],['345',4,5,6,] ,['456',4,5,6,]]
dfs = spark.createDataFrame(s,['id','a','b','c'])
dfs.printSchema()
dfp = spark.createDataFrame(p,['id','a','b','c'])

root
 |-- id: string (nullable = true)
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)
 |-- c: long (nullable = true)



In [53]:
from pyspark.sql.functions import countDistinct, col, lit
def go(colList, df):
    cols = df.columns
    for each in colList:
        if each not in cols:
            df = df.withColumn(each, lit(None))

    return df
unAll = dfs.select((col('id'))).distinct()
# unAll.show()
unqSet = dfp.select(col('id')).distinct()
# unqSet.show()
inter = unAll.subtract(unqSet)
inter.show()
inter = go(dfp.columns,inter)
inter.show()
inter.count()
# dfp.unionByName(inter, allowMissingColumns=True).show()
l = dfp.union(inter)
l.show()

+---+
| id|
+---+
+---+

+---+---+---+---+
| id|  a|  b|  c|
+---+---+---+---+
+---+---+---+---+

+---+---+---+---+
| id|  a|  b|  c|
+---+---+---+---+
|123|  4|  5|  6|
|123|  4|  5|  7|
|234|  4|  5|  6|
|345|  4|  5|  6|
|456|  4|  5|  6|
+---+---+---+---+



In [41]:
dfs.union(dfp).printSchema()

root
 |-- id: string (nullable = true)
 |-- a: string (nullable = true)
 |-- b: string (nullable = true)
 |-- c: long (nullable = true)



In [61]:
sch = {
    'a':'a',
    'x':{'y':['1']}
}

sche = build2(sch)

print(sche)

x = [[1,{'x':{'y':[1,2,3]}}],[2,{'x':{'y':[]}}],[3,{'x':{'y':[]}}]]
s = spark.createDataFrame(x)
s.printSchema()
s.show()

StructType(List(StructField(a,StringType,true),StructField(x,StructType(List(StructField(y,ArrayType(StringType,true),true))),true)))
root
 |-- _1: long (nullable = true)
 |-- _2: map (nullable = true)
 |    |-- key: string
 |    |-- value: map (valueContainsNull = true)
 |    |    |-- key: string
 |    |    |-- value: array (valueContainsNull = true)
 |    |    |    |-- element: long (containsNull = true)

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  1|{x -> {y -> [1, 2...|
|  2|    {x -> {y -> []}}|
|  3|    {x -> {y -> []}}|
+---+--------------------+



In [13]:
df = spark.read.format('csv').option('sep',',').option('header','true').load("/home/sarbajit/Documents/Programs/spark/Data/test.csv")
df.show()

+--------+----+----+----+-------+
|    Name|Roll|Dept|Plan|Details|
+--------+----+----+----+-------+
|Sarbajit|  10| CSE|  A1|      1|
|Shalmoli|  11|  CE|  A1|      2|
|Sarbajit|  10| CSE|  B1|      1|
|Shalmoli|  11|  CE|  A1|      3|
|Shalmoli|  11|  CE|  A2|      4|
|Shalmoli|  11|  CE|  B1|      5|
|Sarbajit|  10|null|  B2|      3|
|Shalmoli|  11|  CE|  A5|      6|
+--------+----+----+----+-------+



In [14]:
# df.take(1)[0].asDict()
from pyspark.sql.functions import count

c= ['Name','Roll','Dept',"Plan","Details"]
df.select(*[ count(each).alias(each) for each in c]).take(1)[0].asDict()


{'Name': 8, 'Roll': 8, 'Dept': 7, 'Plan': 8, 'Details': 8}

In [14]:
from pyspark.sql import Window, functions as func

windowSpec = Window.partitionBy(['Dept','Plan'])
df = df.withColumn('row_count', func.count('Plan').over(windowSpec))
df.show()

+--------+----+----+----+-------+---------+
|    Name|Roll|Dept|Plan|Details|row_count|
+--------+----+----+----+-------+---------+
|Shalmoli|  11|  CE|  A1|      2|        2|
|Shalmoli|  11|  CE|  A1|      3|        2|
|Shalmoli|  11|  CE|  B1|      5|        1|
|Shalmoli|  11|  CE|  A2|      4|        1|
|Sarbajit|  10| CSE|  B2|      3|        1|
|Shalmoli|  11|  CE|  A5|      6|        1|
|Sarbajit|  10| CSE|  A1|      1|        1|
|Sarbajit|  10| CSE|  B1|      1|        1|
+--------+----+----+----+-------+---------+



In [8]:
df1 = spark.createDataFrame([['A',1,2,3],['B',1,5,3],['C',3,10,-3]], schema=['id','id2','b','c'])
df2 = spark.createDataFrame([['A',"XY"],['B','YZ'],['C','ZX']], schema=['id2','x'])
df3 = spark.createDataFrame([['A',"X1"],['B','Y1'],['C','Z1']], schema=['id2','y'])

df1 = df1.join(df2, how="left", on=[df1['id']==df2['id2']]).drop(df2["id2"])
df1.join(df3, how="left", on=[df1['id']==df3['id2']]).drop(df3["id2"]).show()

+---+---+---+---+---+---+
| id|id2|  b|  c|  x|  y|
+---+---+---+---+---+---+
|  B|  1|  5|  3| YZ| Y1|
|  C|  3| 10| -3| ZX| Z1|
|  A|  1|  2|  3| XY| X1|
+---+---+---+---+---+---+



In [1]:
spark.stop()

NameError: name 'spark' is not defined